In [1]:
!pip install ray

     |████████████████████████████████| 47.5MB 82kB/s 
     |████████████████████████████████| 3.2MB 42.8MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 133kB 45.0MB/s 
     |████████████████████████████████| 204kB 48.0MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 1.3MB 40.8MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 143kB 58.0MB/s 
     |████████████████████████████████| 296kB 55.8MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=7478642c9df045809e52d068d26e4540c370bee1fbb7184ed2266459fa1fbc2a
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat


In [4]:
import ray
import time
ray.init()

RuntimeError: ignored

## ray

remote function은 Ray의 프로세스에 의해 비동기적으로 실행됩니다. 따라서 아래의 코드는 ray에 의해서 비동기적으로 실행됩니다.

In [5]:
@ray.remote(num_cpus=0.5)
def f(x):
    time.sleep(1)
    return x

In [6]:
start = time.time()
# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))

# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(result_ids)  # [0, 1, 2, 3]

print("Ray 소요시간: {}".format(time.time() - start))

Ray 소요시간: 2.014763593673706


In [7]:
def f(x):
    time.sleep(1)
    return x

In [8]:
start = time.time()
result_ids = [] 
for i in range(4):
    result_ids.append(f(i))
print("소요시간 : {}".format(time.time() - start))

소요시간 : 4.004309415817261


## Task in Task

병렬로 x_id와 y_id를 연산하고 z_id를 구하는 예제

In [9]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

size = 10000

# Get the results with ray.
start = time.time()
x_id = create_matrix.remote([size, size])
y_id = create_matrix.remote([size, size])
z_id = multiply_matrices.remote(x_id, y_id)
z = ray.get(z_id)
print("z_id_with_ray : {}".format(time.time() - start))

# Get the results with np array.
start = time.time()
x_id = np.random.normal([size, size])
y_id = np.random.normal([size, size])
z_id = np.dot(x_id, y_id)
print("z_id_with_np_array : {}".format(time.time() - start))

z_id_with_ray : 70.33507108688354
z_id_with_np_array : 0.0007925033569335938


## Aggregation

연산을 어떻게 병렬로 배치하느냐에 따라서 보는 속도의 차이
![](https://miro.medium.com/max/1400/1*vHz3troEmr4uLns0V8VmdA.jpeg)


In [10]:
import time

@ray.remote
def add(x, y):
    time.sleep(1)
    return x + y

In [11]:
start = time.time()
# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)
print("Vanilla version : {}".format(time.time() - start))


start = time.time()
# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)
print("Advanced version : {}".format(time.time() - start))

Vanilla version : 7.02429461479187
Advanced version : 4.013916254043579


## Reference

https://towardsdatascience.com/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray-99f8d70369b8